lan a 5-day Goa trip with budget under 30k and current weather.”

That requires:

Search

Weather

Budget reasoning

Structured output

In [7]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.tools import tool
import requests
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY

In [8]:
from langchain_community.tools import DuckDuckGoSearchRun
search_result = DuckDuckGoSearchRun()
llm = ChatGoogleGenerativeAI(model='gemini-2.5-flash',temperature=0)

In [9]:
@tool
def get_weather_details(city : str) -> str:
    """
        this function tells the 7 days weather of a city
    """
    url = f"http://api.weatherapi.com/v1/forecast.json?key=8d6c75fe91d548749b763632262202&q={city}&days=7"
    response = requests.get(url)
    data = response.json()
    forecast_days = data["forecast"]["forecastday"]
    
    result = f"7-Day Weather Forecast for {city}:\n"
    
    for day in forecast_days:
        date = day["date"]
        condition = day["day"]["condition"]["text"]
        temp = day["day"]["avgtemp_c"]
        result += f"{date}: {condition}, Avg Temp: {temp}°C\n"
    
    return result

In [10]:
from langgraph.prebuilt import create_react_agent
agent = create_react_agent(
    model = llm,
    tools = [search_result, get_weather_details]
)

C:\Users\HP\AppData\Local\Temp\ipykernel_13524\2415814333.py:2: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(


In [11]:
# Function to extract total cost
import re # re->regex
def extract_total_cost(text):
    match = re.search(r"TOTAL_COST:\s*(\d+)", text)
    if match:
        return int(match.group(1))
    return None


MAX_BUDGET = 30000

for attempt in range(3):

    response = agent.invoke({
        "messages": [
            ("user", """
            Plan a 7-day trip to Goa, India.
Budget must be under ₹30,000.

Follow these steps strictly:

1. First, call the get_weather_details tool for "Goa".
2. Print the 7-day weather clearly (date-wise).
3. Then create a 7-day trip plan.
4. For each day include:
   - Activities
   - Estimated cost for that day
   - Weather for that specific date
5. At the end calculate total cost properly.

At the end write strictly in this format:
TOTAL_COST: <number only>

Do not skip weather tool.
            """)
        ]
    })
    #extraction
    output_text = response['messages'][-1].content[0]['text']
    total_cost = extract_total_cost(output_text)
    if total_cost is not None and total_cost <= MAX_BUDGET:
        print("✅ Valid Plan Found\n")
        print(output_text)
        break
    else:
        print("❌ Budget exceeded. Regenerating...\n")  

✅ Valid Plan Found

**7-Day Weather Forecast for Goa:**

*   **2026-02-22:** Sunny, Avg Temp: 26.5°C
*   **2026-02-23:** Sunny, Avg Temp: 26.5°C
*   **2026-02-24:** Sunny, Avg Temp: 25.8°C
*   **2026-02-25:** Sunny, Avg Temp: 25.4°C
*   **2026-02-26:** Sunny, Avg Temp: 25.0°C
*   **2026-02-27:** Sunny, Avg Temp: 25.0°C
*   **2026-02-28:** Sunny, Avg Temp: 25.0°C

---

**7-Day Trip Plan to Goa (Budget under ₹30,000)**

**Day 1: Arrival in North Goa & Beach Relaxation**
*   **Activities:** Arrive at Goa International Airport (GOI), take a taxi/bus to a pre-booked budget guesthouse in Anjuna/Vagator. Check-in and relax. Head to Anjuna Beach for sunset and dinner at a beach shack.
*   **Estimated Cost:** ₹3500
*   **Weather:** Sunny, Avg Temp: 26.5°C

**Day 2: North Goa Exploration - Beaches & Markets**
*   **Activities:** Rent a scooter. Explore Vagator Beach, Chapora Fort for panoramic views. Visit Anjuna Flea Market (if Wednesday) or local shops. Enjoy water sports (optional, extra cost

develop constraint-aware AI travel planning agent with tool integration

Built a multi-tool AI agent using LangGraph and Gemini (gemini-2.5-flash)
to generate structured 7-day Goa trip plans under ₹30,000 budget.

Key Features:
- Integrated DuckDuckGo search tool for real-time data retrieval
- Created custom WeatherAPI tool for 7-day forecast integration
- Used ReAct agent architecture for reasoning + tool execution
- Enforced structured output with strict TOTAL_COST formatting
- Implemented regex-based cost extraction
- Added hard budget constraint validation logic
- Enabled regeneration loop if total cost exceeds limit
- Designed clear day-wise output including weather + cost breakdown

This system demonstrates tool usage, API integration, structured parsing,
and constraint-based regeneration in an AI agent workflow.